In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys
from pyprojroot import here
from pathlib import Path

module_path = here()
if module_path not in sys.path:
    sys.path.append(str(module_path.resolve()))

import bayes_opt as bo
from mf2 import forrester
import multiLevelCoSurrogates as mlcs
from sklearn.gaussian_process import GaussianProcessRegressor, kernels

np.random.seed(20160501)  # Setting seed for reproducibility

In [ ]:
import warnings
from scipy.stats import norm

class UtilityFunction:
    """
    An object to compute the acquisition functions.
    """

    def __init__(self, kind, kappa=2.576, xi=1, kappa_decay=1, kappa_decay_delay=0):

        self.kappa = kappa
        self.xi = xi

        self._kappa_decay = kappa_decay
        self._kappa_decay_delay = kappa_decay_delay
        self._iters_counter = 0

        if kind not in ['ucb', 'ei', 'ei_orig', 'poi']:
            err = "The utility function " \
                  "{} has not been implemented, " \
                  "please choose one of ucb, ei, or poi.".format(kind)
            raise NotImplementedError(err)
        else:
            self.kind = kind

    def update_params(self):
        self._iters_counter += 1

        if self._kappa_decay < 1 and self._iters_counter > self._kappa_decay_delay:
            self.kappa *= self._kappa_decay

    def utility(self, x, gp, y_best, goal='maximize'):
        if self.kind == 'ucb':
            return self._ucb(x, gp, self.kappa, goal)
        if self.kind == 'ei':
            return self._ei(x, gp, y_best, self.xi, goal)
        if self.kind == 'ei_orig':
            return self._ei_orig(x, gp, y_best, self.xi)
        if self.kind == 'poi':
            return self._poi(x, gp, y_best, self.xi, goal)

    @staticmethod
    def _ucb(x, gp, kappa, goal):
        with warnings.catch_warnings():
           warnings.simplefilter("ignore")
           mean, std = gp.predict(x, return_std=True)
        if goal == 'maximize':
            return mean + kappa * std
        elif goal == 'minimize':
            return mean - kappa * std

    @staticmethod
    def _ei(x, gp, y_best, xi, goal):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            mean, std = gp.predict(x, return_std=True)
  
        if goal == 'maximize':
            a = (mean - y_best - xi)
        elif goal == 'minimize':
            a = (y_best - mean + xi)

        z = a / std
#         print(f'{a=}, {z=}')
        if goal == 'maximize':
            return a * norm.cdf(z) + std * norm.pdf(z)
        elif goal == 'minimize':
            return a * norm.cdf(z) - std * norm.pdf(z)

    @staticmethod
    def _ei_orig(x, gp, y_max, xi):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            mean, std = gp.predict(x, return_std=True)
  
        a = (mean - y_max - xi)
        z = a / std
        return a * norm.cdf(z) + std * norm.pdf(z)

    @staticmethod
    def _poi(x, gp, y_best, xi, goal):
        with warnings.catch_warnings():
           warnings.simplefilter("ignore")
           mean, std = gp.predict(x, return_std=True)
        if goal == 'maximize':
            z = (mean - y_best - xi)/std
        elif goal == 'minimize':
            z = (y_best - mean - xi)/std
        return norm.cdf(z)

In [ ]:
def acq_max(ac, gp, y_max, bounds, random_state, n_warmup=1000, n_iter=10):
    """
    A function to find the maximum of the acquisition function
    It uses a combination of random sampling (cheap) and the 'L-BFGS-B'
    optimization method. First by sampling `n_warmup` (1e5) points at random,
    and then running L-BFGS-B from `n_iter` (250) random starting points.
    Parameters
    ----------
    :param ac:
        The acquisition function object that return its point-wise value.
    :param gp:
        A gaussian process fitted to the relevant data.
    :param y_max:
        The current maximum known value of the target function.
    :param bounds:
        The variables bounds to limit the search of the acq max.
    :param random_state:
        instance of np.RandomState random number generator
    :param n_warmup:
        number of times to randomly sample the aquisition function
    :param n_iter:
        number of times to run scipy.minimize
    Returns
    -------
    :return: x_max, The arg max of the acquisition function.
    """

    # Warm up with random points
    x_tries = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                   size=(n_warmup, bounds.shape[0]))
    print(x_tries.shape)
    ys = ac(x_tries, gp=gp, y_best=y_max)
    print(ys.shape)
    x_max = x_tries[ys.argmax()]
    max_acq = ys.max()

    # Explore the parameter space more throughly
    x_seeds = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                   size=(n_iter, bounds.shape[0]))
    for x_try in x_seeds:
        # Find the minimum of minus the acquisition function
        res = minimize(lambda x: -ac(x.reshape(1, -1), gp=gp, y_best=y_max),
                       x_try.reshape(1, -1),
                       bounds=bounds,
                       method="L-BFGS-B")

        # See if success
        if not res.success:
            continue

        # Store it if better than previous minimum(maximum).
        if max_acq is None or -res.fun[0] >= max_acq:
            x_max = res.x
            max_acq = -res.fun[0]

    # Clip output to make sure it lies within the bounds. Due to floating
    # point technicalities this is not always the case.
    return np.clip(x_max, bounds[:, 0], bounds[:, 1])

In [ ]:
plot_x = np.linspace(start=0,stop=1,num=51).reshape(-1,1)
plot_high = forrester.high(plot_x).reshape(-1, 1)

high_x = np.array([0, 0.4, 0.6, 1]).reshape(-1,1)
high_y = forrester.high(high_x).reshape(-1, 1)

gp_direct = GaussianProcessRegressor()
gp_direct.fit(high_x, high_y)



line, = plt.plot(plot_x, plot_high, label='high')
plt.scatter(high_x, high_y, color=line.get_color())
plt.plot(plot_x, gp_direct.predict(plot_x), label='high-fit GP')
plt.legend(loc=1)
plt.show()

In [ ]:
y_best = min(high_y)
ac = UtilityFunction(kind='ei_orig').utility
# plt.plot(UtilityFunction(kind='ei_orig').utility(plot_x, gp=gp_direct, y_best=y_best))
# plt.plot(UtilityFunction(kind='ei').utility(plot_x, gp=gp_direct, y_best=y_best, goal='maximize'))
plt.plot(UtilityFunction(kind='ei').utility(plot_x, gp=gp_direct, y_best=y_best, goal='minimize'))
plt.show()

In [ ]:
res = acq_max(ac, gp_direct, y_best, forrester.bounds.T, np.random.RandomState())